In [1]:
import json
import random
from PIL import Image
from lib import image_augments as aug
from lib.file_utils import create_directory
from lib.images import list_images
from lib import image_augments as aug
from pascal import annotation_from_xml
from pascal.utils import save_xml

In [2]:
DATASET_DIR = "./dataset"
IMAGE_DIR = "./images"

with open(f'{DATASET_DIR}/catalog.json') as json_file:
    catalog = json.load(json_file)
print(catalog)

{'MM': 'medical-mask', 'PM': 'person-mask', 'PMM': 'person-masked-medical', 'PMN': 'person-masked-non-medical', 'MIXED': 'mixed'}


## Define Augmentations

In [3]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

def random_augmentation(img):
    augments = []
    aug_type = random.randint(0, 5)
    if aug_type == 1: augments.extend(aug.rgb_shift(img, 15))
    if aug_type == 2: augments.extend(aug.channel_shuffle(img))
    if aug_type == 3: augments.extend(aug.channel_shuffle(img))
    if aug_type == 4: augments.extend(aug.enhancements(img, factors=(0.5, 0.5, 0.1, 0.3)))
    if aug_type == 5: augments.extend(aug.blur(img, radius=3))
    return augments

In [4]:
def read_pascal_voc(image_path):
    end_index = image_path.rindex('.')
    xml_path = image_path[0:end_index] + '.xml'
    annotations = annotation_from_xml(xml_path)
    return annotations

def copy_annotation_file(image_path, path_annotation_file):
    pascal_voc = read_pascal_voc(image_path)
    xml = pascal_voc.to_xml()
    save_xml(path_annotation_file, xml)
    #with open(path_annotation_file, 'w') as file:
    #    file.write(str(pascal_voc))

In [5]:
IMAGE_AUG_DIR = "./images-aug"
for class_dir in catalog.keys():
    create_directory(f'{IMAGE_AUG_DIR}/train/{class_dir}')
    print(f'Processing {class_dir}...')
    
    image_paths = list_images(f'{IMAGE_DIR}/train/{class_dir}')
    for image_path in image_paths:
        img = Image.open(image_path)
        augments = random_augmentation(img)
        for _, [suffix, aug_img] in enumerate(augments):
            file_name, file_ext = image_path.split("/")[-1].split(".")
            aug_file_path = f'{IMAGE_AUG_DIR}/train/{class_dir}/{file_name}_{suffix}'
            aug_img.save(f'{aug_file_path}.{file_ext}')
            copy_annotation_file(image_path, f'{aug_file_path}.xml')

Processing MM...
Processing PM...
Processing PMM...
Processing PMN...
Processing MIXED...
